<a href="https://colab.research.google.com/github/ncrncec23/PBT-Face-recognition/blob/main/PBT_FaceRecognitonModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import standard dependencies / Uvoz standardnih knjižica
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import uuid

# Import tensorflow dependencies - Functional API / Uvoz tensorflow knjižica konkretno Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf


In [5]:
# Avoid OOM (Out of memory) errors by setting GPU Memory Consumption Growth
# Sprječavanje OOM grešaka postavljanjem ograničenja za korištenje GPU memorije.
def GpuGrowth():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
def Preprocess(file_path):

    # Read in the image / Pročitaj sliku
    byte_img = tf.io.read_file(file_path)
    # Load in the image / Učitaj sliku
    img = tf.io.decode_jpeg(byte_img)

    # Preprocessing steps / Koraci obrade
    img = tf.image.resize(img, (100, 100))
    # Scale the image to [0,1] / Promijeni veličinu slike na [0,1]
    img = img / 255.0

    # Return image
    return img

In [7]:
# Preprocess the images in the Dataset / Obradi slike u skupu podataka
def Preprocess_Twin(input_img, validation_img, label):
    return(Preprocess(input_img), Preprocess(validation_img), label)

In [8]:
def make_embedding():
    inp = Input(shape=(100, 100, 3), name='input_image')

    # First block of convolutional layers / Prva blok konvolucijskih slojeva
    c1 = Conv2D(64, (10, 10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2, 2), padding='same')(c1)

    # Second block of convolutional layers / Druga blok konvolucijskih slojeva
    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2, 2), padding='same')(c2)

    # Third block of convolutional layers / Treća blok konvolucijskih slojeva
    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2, 2), padding='same')(c3)

    # Final embedding block / Završni blok
    c4 = Conv2D(256, (4, 4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=inp ,outputs=d1, name='Embedding')

In [9]:
# Siamise L1 distance Class / Siamise L1 udaljenost klasa
class L1Dist(Layer):

    # Init method / Inicijalizacija
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # Similarity method / Metoda sličnosti
    def call(self, inputs):
        input_embedding, validation_embedding = inputs
        return tf.math.abs(input_embedding - validation_embedding)

In [11]:
def make_siamese_model():

    # Anchor input image in the network / Sidrova ulazna slika u mreži
    input_image = Input(name='input_image', shape=(100, 100, 3))

    # Validation input image in the network / Ulazna slika za validaciju u mreži
    validation_image = Input(name='validation_image', shape=(100, 100, 3))

    # Combine siamese distance components / Kombiniraj komponente siamiskih udaljenosti
    siamese_layer = L1Dist(name='distance')
    distances = siamese_layer([embedding(input_image), embedding(validation_image)])

    # Classification layer / Klasa klasifikacije
    classifier = Dense(1, name='dense', activation='sigmoid')(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [31]:
GpuGrowth()
POS_PATH = os.path.join('/content/drive/MyDrive/ColabNotebooks/data', 'positive')
NEG_PATH = os.path.join('/content/drive/MyDrive/ColabNotebooks/data', 'negative')
ANC_PATH = os.path.join('/content/drive/MyDrive/ColabNotebooks/data', 'anchor')

In [32]:
anchor = tf.data.Dataset.list_files(ANC_PATH +'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH +'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH +'/*.jpg').take(300)

dir_test = anchor.as_numpy_iterator()

In [37]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [39]:
# Bulid dataloader pipeline / Izgradi pipeline za učitavanje podataka
data = data.map(Preprocess_Twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [40]:
# Training partition / Particija za treniranje
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# Testing partition / Particija za testiranje
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Embedding model / Ugradbeni model
embedding = make_embedding()
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_image    │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Embedding           │ (None, 4096)      │ 38,960,448 │ input_image[0][0… │
│ (Functional)        │                   │            │ validation_image… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ distance (L1Dist)   │ (None, 4096)      │          0 │ Embedding[0][0],  │
│                     │                   │            │ Embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │      4,097 │ distance[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Setup loss function and optimizer / Konfiguracija funkcije gubitka i optimizatora
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4)

# Establish the checkpoint directory / Uspostavi direktorij za checkpoint
checkpoint_dir = 'drive/MyDrive/ColabNotebooks/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [53]:
@tf.function
def train_step(batch):

    with tf.GradientTape() as tape:

        # Get anchor and positive/negative images / Uzimanje sidrenih i pozitivnih/negativnih slika
        X = batch[:2]
        # Get the labels / Uzimanje oznaka
        Y = batch[2]

        # Foward pass / Prolaz unaprijed
        yhat = siamese_model(X, training=True)
        # Calculate the loss / Izračunaj gubitak
        loss = binary_cross_entropy(Y, yhat)

    # Calculate gradients / Izračunaj gradijente
    grad = tape.gradient(loss, siamese_model.trainable_variables)

    # Calculate updated weights and apply to siamese model / Izračunaj ažurirane težine i primijeni na siamiskom modelu
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))

    # Return loss / Vratiti gubitak
    return loss

# Training loop / Petlja treniranja
def train(data, EPOCHS):
    # Loop through epochs / Petlja kroz epohe
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        # Loop through each batch / Petlja kroz svaki batch
        for idx, batch in enumerate(data):
            # Run train step here
            train_step(batch)
            progbar.update(idx+1)

        # Save checkpoints / Spremi checkpoint
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [54]:
# Train the model / Treniraj model
EPOCHS = 50
train(train_data, EPOCHS)


 Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 190ms/step

 Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 201ms/step

 Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step

 Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 202ms/step

 Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 202ms/step

 Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step

 Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step

 Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step

 Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step

 Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 200ms/step

 Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 199ms/step

 Epoch 12/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 199ms/step

 Epoch 13/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 198ms/step

 Epoch 14/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step

 Epoch 15/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step

 Epoch 16/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step

 Epoch 17/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step

 Epoch 18/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step

 Epoch 19/50
19/1

In [61]:
# Import metric calcuations / Uvoz metrika
from tensorflow.keras.metrics import Precision, Recall

# Get a batch of test data / Uzimanje testnog skupa podataka
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

# Make predictions / Predikcija
y_hat = siamese_model.predict([test_input, test_val])

# Post processing the results / Postprocesiranje rezultata
y_pred = [1 if prediction > 0.5 else 0 for prediction in y_hat ]

# Calculate the FAR and FRR / Izračunaj FAR i FRR
false_accepts = np.logical_and(y_pred == 1, y_true == 0)
false_rejects = np.logical_and(y_pred == 0, y_true == 1)
FAR = np.sum(false_accepts) / np.sum(y_true == 0)
FRR = np.sum(false_rejects) / np.sum(y_true == 1)

# Print the results / Ispis rezultata
print(f"FAR: {FAR * 100:.2f}%")
print(f"FRR: {FRR * 100:.2f}%")

# Calculate precision and recall / Izračunaj preciznost i odziv
m = Recall()

# Calculate the recall / Izračunaj odziv
m.update_state(y_true, y_hat)

# Return recall result / Vratiti rezultat odziva
m.result().numpy()

# Creating a metric object
m = Precision()

# Calculating the recall value
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
FAR: 0.00%
FRR: 0.00%


np.float32(1.0)

In [69]:
# Save the model / Spremi model
siamese_model.save('siamesemodel.h5')

In [70]:
model = tf.keras.models.load_model('siamesemodel.h5',
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [71]:
model.predict([test_input, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 843ms/step


array([[9.9999881e-01],
       [6.2373599e-11],
       [1.0814385e-09],
       [1.0000000e+00],
       [9.9862969e-01],
       [2.8462168e-06],
       [9.9995780e-01],
       [1.0000000e+00],
       [2.2066404e-08],
       [9.9989080e-01],
       [9.9999690e-01],
       [1.3858434e-08],
       [9.9999905e-01],
       [4.7694352e-06],
       [1.8513490e-09],
       [2.6024829e-09]], dtype=float32)